In [2]:
import numpy as np
from pc_kriging import PC_Kriging
import matplotlib.pyplot as plt
from scipy.stats import norm
from doepy import build
from scipy import optimize
import pandas as pd
from numpy import genfromtxt
import pickle

from scipy.special import eval_legendre, eval_hermitenorm
import scipy.special

# adaptive learning - expected feasiability function --------------------------------------------------------------

def EFF(u,v,z):
    zl=-2*v
    zh=2*v
    return ((u-z)*( 2*norm.cdf((z-u)/v) - norm.cdf((zl-u)/v) - norm.cdf((zh-u)/v)) 
           -(v)*( 2*norm.pdf((z-u)/v) - norm.pdf((zl-u)/v) - norm.pdf((zh-u)/v))  
           +(2*v)*(norm.cdf((zh-u)/v) - norm.cdf((zl-u)/v)))

def U_function(u, v):
    return np.abs(u)/v

def LinearNorm(x,oldmin,oldmax,newmin,newmax):    # scaling linearly X to new domain limits
    return newmin + ((x-oldmin)*(newmax-newmin)/(oldmax-oldmin))


def VoronoiCell(x,xn):   #given x [single value] return the index of the closest xn [array]
    dist=PCK1.distance(x.reshape(1,-1),xn)
    return np.argmin(dist)

# Limit State 1

In [3]:
# #ground truth function 1 ----------------------------------------------------
# https://rprepo.readthedocs.io/en/latest/reliability_problems.html#rp201

def gfun_53(x):
    """Performance function for reliability problem 53.

    Parameters
    ----------
        x : numpy.array of float(s)
            Values of independent variables: columns are the different parameters/random variables (x1, x2,...xn) and rows are different parameter/random variables sets for different calls.

    Returns
    -------
        g_val_sys : numpy.array of float(s)
            Performance function value for the system.
        g_val_comp : numpy.array of float(s)
            Performance function value for each component.
        msg : str
            Accompanying diagnostic message, e.g. warning.
    """
#     import numpy as np
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(5*x[:, 0]/2) + 2 - (x[:, 0]**2 + 4)*(x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

# Limit State 2

In [4]:
# ground truth function 2 ----------------------------------------------------
# https://rprepo.readthedocs.io/en/latest/reliability_problems.html#rp201

def LState2(x):
    #"""Modified Performance function for reliability problem 53""""
  
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(2.0*x[:, 0]) -0.5 - (x[:, 0]**2 + 4)*(-1.0*x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

In [5]:
config = {"pol_type": ['hermite', 'hermite']}   #design variables following normal distribution
PCK1 = PC_Kriging(config)
PCK_loo = PC_Kriging(config)    # for LOOCV with same 'config' as specified in the original model

## Initial training 

In [6]:
function_1 = gfun_53   #ground truth function 1 
function_2 = LState2   #ground truth function 2

dim = 2       # dimensionality

x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
x2mean, x2sigma = 2.5 , 1.0  # normal distribution 

ntest = 5000  # test points

# TEST POINTS -------------------------------------------------
XR = np.zeros((int(ntest), dim))   #normalized test points
XN = np.zeros((int(ntest), dim))  #scaled test points
YN_1 = np.zeros(int(ntest))
YN_2 = np.zeros(int(ntest))
#variable 1 ---------------------------------------------------
XN[:,0] = np.random.normal(0,1,ntest)  
XR[:,0] = PCK1.scalehermite(XN[:,0], x1mean, x1sigma)  
#variable 2 ---------------------------------------------------
XN[:,1] = np.random.normal(0,1,ntest)  
XR[:,1] = PCK1.scalehermite(XN[:,1], x2mean, x2sigma)  

YN_1 = function_1(XR)[0]
YN_2 = function_2(XR)[0]

In [7]:
# objective function to optimize length scale --------------------------------------------------------
def L_Object (l):
    v = 5/2
    N = len(xn)
    R = PCK1.matern(xn , xn, l, v)
    detR = np.linalg.det(R)
    
    modelpar2 = PCK1.train(xn, yn, p, np.array([l,v]))    # returns B, sig2, InfoMatrix(phi) , PolyIndices(alpha)
    ### ------------------Theta_ by UQLab User Manual PCK(C. Lataniotis, D. Wicaksono, S. Marelli, B. Sudret)------------------------------
    sig2 = modelpar2[1].reshape(-1)
    # return 0.5*(np.log(detR)+ N*np.log(2*np.pi*sig2)+ N)

    ### ------------------Theta_ by MLE PCK(Schobi,Sudret,Wiart)------------------------------
    FB = PCK1.InfoMat @ modelpar2[0]
    ins = (yn-FB).reshape(-1)
    R_1 = np.linalg.inv(R)
    return ((ins.T) @ R_1 @ ins) * (1/N) * (detR**(1/N))

In [ ]:
fail_samples_1 = np.sum(YN_1 < 0 )
Pf_ref_1 = fail_samples_1/ntest

fail_samples_2 = np.sum(YN_2 < 0 )
Pf_ref_2 = fail_samples_2/ntest

Pf_ref_1, Pf_ref_2

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(XR[:,0],XR[:,1], YN_1)
ax.scatter(XR[:,0],XR[:,1], YN_2)             #plotting the last trained surrogate
# ax.scatter(xr[:,0],xr[:,1], yn, color='red')   #observation points 
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('G_System')

# Combined Active training - U - 2 LS

In [8]:
number_experiments = 15    
number_active_points = 40

results_file = '_CATSeq_U_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        if ( len(xn) % 2 == 0 ):   #even number
            meanMC = meanMC_1
            varMC = varMC_1                 
            print ('LS 1')

        else:    #odd number
            meanMC = meanMC_2
            varMC = varMC_2
            print ('LS 2')

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), varMC.reshape(-1))
        
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


LS1:  Degree 1 e_LOO 0.018485882672114946 Pf_ref 0.03094 Pf 0.05079 CoV 0.01370
LS2:  Degree 1 e_LOO 0.0011009662988686775 Pf_ref 0.14879 Pf 0.19036 CoV 0.00650
LS 1
number of training points:  11 -------------------------------------------------


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:137: RuntimeWarning: divide by zero encountered in double_scalars


LS1:  Degree 2 e_LOO 0.0076854095263916644 Pf_ref 0.03031 Pf 0.0 CoV inf
LS2:  Degree 3 e_LOO 0.005850160809434901 Pf_ref 0.14975 Pf 0.12897 CoV 0.00820
LS 2
number of training points:  12 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.00028749279064677714 Pf_ref 0.03165 Pf 0.0 CoV inf
LS2:  Degree 2 e_LOO 0.017579782724030827 Pf_ref 0.14853 Pf 0.15775 CoV 0.00730
LS 1
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.00850682223552661 Pf_ref 0.03071 Pf 0.00053 CoV 0.13730
LS2:  Degree 1 e_LOO 0.0256880632213963 Pf_ref 0.14672 Pf 0.13064 CoV 0.00820
LS 2
number of training points:  14 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0019382810572298236 Pf_ref 0.03126 Pf 0.00073 CoV 0.11700
LS2:  Degree 2 e_LOO 2.7231089459750528e-05 Pf_ref 0.14815 Pf 0.14244 CoV 0.00780
LS 1
number of training points:  15 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.000610

...

LS2:  Degree 2 e_LOO 0.00018762136168752418 Pf_ref 0.14792 Pf 0.10748 CoV 0.00910
LS 2
number of training points:  24 -------------------------------------------------
LS1:  Degree 1 e_LOO 9.726594594088907e-05 Pf_ref 0.03156 Pf 0.00745 CoV 0.03650
LS2:  Degree 4 e_LOO 0.0016365270797787672 Pf_ref 0.14821 Pf 0.18164 CoV 0.00670
LS 1
number of training points:  25 -------------------------------------------------
LS1:  Degree 1 e_LOO 1.5891006681461978e-05 Pf_ref 0.03092 Pf 0.00851 CoV 0.03410
LS2:  Degree 2 e_LOO 7.134468609111147e-05 Pf_ref 0.14607 Pf 0.13178 CoV 0.00810
LS 2
number of training points:  26 -------------------------------------------------
LS1:  Degree 1 e_LOO 3.703598154574974e-05 Pf_ref 0.03187 Pf 0.00842 CoV 0.03430
LS2:  Degree 2 e_LOO 6.567182679180362e-05 Pf_ref 0.14843 Pf 0.13365 CoV 0.00810
LS 1
number of training points:  27 -------------------------------------------------
LS1:  Degree 1 e_LOO 2.3025437789306505e-06 Pf_ref 0.03162 Pf 0.00913 CoV 0.03290
LS2: 

LS2:  Degree 3 e_LOO 0.0004101383129639761 Pf_ref 0.14954 Pf 0.12951 CoV 0.00820
LS 1
number of training points:  17 -------------------------------------------------
LS1:  Degree 2 e_LOO 3.569959174212342e-06 Pf_ref 0.03205 Pf 0.00509 CoV 0.04420
LS2:  Degree 3 e_LOO 0.014273884775829046 Pf_ref 0.14891 Pf 0.10644 CoV 0.00920
LS 2
number of training points:  18 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.00031953978687061584 Pf_ref 0.03199 Pf 0.00506 CoV 0.04430
LS2:  Degree 2 e_LOO 0.006854789070913374 Pf_ref 0.1473 Pf 0.10404 CoV 0.00930
LS 1
number of training points:  19 -------------------------------------------------
LS1:  Degree 2 e_LOO 7.371145243532361e-05 Pf_ref 0.03065 Pf 0.00913 CoV 0.03290
LS2:  Degree 3 e_LOO 0.0024147348924945565 Pf_ref 0.1495 Pf 0.12817 CoV 0.00820
LS 2
number of training points:  20 -------------------------------------------------
LS1:  Degree 2 e_LOO 8.246624450406109e-05 Pf_ref 0.03184 Pf 0.00926 CoV 0.03270
LS2:  Degre

LS1:  Degree 2 e_LOO 0.0001009787397168042 Pf_ref 0.03115 Pf 0.10343 CoV 0.00930
LS2:  Degree 1 e_LOO 2.545247658294239e-07 Pf_ref 0.14624 Pf 0.19162 CoV 0.00650
LS 1
number of training points:  11 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0002498489046863664 Pf_ref 0.03223 Pf 0.09715 CoV 0.00960
LS2:  Degree 3 e_LOO 1.4519140002273974e-05 Pf_ref 0.14798 Pf 0.19083 CoV 0.00650
LS 2
number of training points:  12 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.0007160716661921955 Pf_ref 0.03083 Pf 0.08096 CoV 0.01070
LS2:  Degree 3 e_LOO 3.053596940202548e-06 Pf_ref 0.147 Pf 0.19903 CoV 0.00630
LS 1
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0017851992510390683 Pf_ref 0.03219 Pf 0.04731 CoV 0.01420
LS2:  Degree 2 e_LOO 0.0010427722415055746 Pf_ref 0.14734 Pf 0.25021 CoV 0.00550
LS 2
number of training points:  14 -------------------------------------------------
LS1:  Deg

LS2:  Degree 2 e_LOO 4.288308811219276e-06 Pf_ref 0.14749 Pf 0.14627 CoV 0.00760
LS 2
number of training points:  44 -------------------------------------------------
LS1:  Degree 4 e_LOO 2.9950149231684206e-05 Pf_ref 0.03056 Pf 0.01716 CoV 0.02390
LS2:  Degree 2 e_LOO 2.935034287137155e-10 Pf_ref 0.14869 Pf 0.14776 CoV 0.00760
LS 1
number of training points:  45 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.00012576468620387456 Pf_ref 0.03141 Pf 0.01771 CoV 0.02360
LS2:  Degree 2 e_LOO 4.4064709242663335e-06 Pf_ref 0.14835 Pf 0.14737 CoV 0.00760
LS 2
number of training points:  46 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0012171943758353472 Pf_ref 0.0318 Pf 0.01534 CoV 0.02530
LS2:  Degree 2 e_LOO 0.00016675483356624988 Pf_ref 0.14872 Pf 0.14959 CoV 0.00750
LS 1
number of training points:  47 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0014502799969411616 Pf_ref 0.03093 Pf 0.02941 CoV 0.01820
LS2: 

LS2:  Degree 2 e_LOO 1.0730439866363225e-06 Pf_ref 0.14838 Pf 0.15053 CoV 0.00750
LS 1
number of training points:  37 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0009229656700313332 Pf_ref 0.03113 Pf 0.02256 CoV 0.02080
LS2:  Degree 2 e_LOO 8.775699325244057e-05 Pf_ref 0.1473 Pf 0.14715 CoV 0.00760
LS 2
number of training points:  38 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.00115773002810863 Pf_ref 0.03061 Pf 0.0226 CoV 0.02080
LS2:  Degree 4 e_LOO 0.0001197115508917832 Pf_ref 0.14888 Pf 0.15843 CoV 0.00730
LS 1
number of training points:  39 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.000966701898275282 Pf_ref 0.03055 Pf 0.02241 CoV 0.02090
LS2:  Degree 4 e_LOO 3.2876769944935216e-05 Pf_ref 0.14988 Pf 0.15969 CoV 0.00730
LS 2
number of training points:  40 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0007410477555317189 Pf_ref 0.03151 Pf 0.02299 CoV 0.02060
LS2:  Degre

# Combined training - EFF - 2 LS

In [8]:
number_experiments = 15
number_active_points = 40

results_file = '_CATSeq_EFF_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        if ( len(xn) % 2 == 0 ):   #even number
            meanMC = meanMC_1
            varMC = varMC_1                 
            print ('LS 1')

        else:    #odd number
            meanMC = meanMC_2
            varMC = varMC_2
            print ('LS 2')
        
        ### Evaluating new points
        eff = EFF(meanMC.reshape(-1), varMC.reshape(-1) , 0 )  #(mean, variance, target) 
        
        xr = np.append(xr, MCinputs[np.argmax(eff)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


LS1:  Degree 1 e_LOO 3.836931236001603e-06 Pf_ref 0.032 Pf 0.06413 CoV 0.01210
LS2:  Degree 1 e_LOO 4.240468345252064e-05 Pf_ref 0.14862 Pf 0.16308 CoV 0.00720
number of training points:  11 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.046620807954644565 Pf_ref 0.03198 Pf 0.13326 CoV 0.00810
LS2:  Degree 2 e_LOO 0.03357610546351106 Pf_ref 0.14861 Pf 0.13187 CoV 0.00810
number of training points:  12 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.016214618248466598 Pf_ref 0.03123 Pf 0.13193 CoV 0.00810
LS2:  Degree 2 e_LOO 0.08277592172385313 Pf_ref 0.15097 Pf 0.11813 CoV 0.00860
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0392053227429953 Pf_ref 0.0312 Pf 0.05091 CoV 0.01370
LS2:  Degree 1 e_LOO 0.002578679943607183 Pf_ref 0.14773 Pf 0.1142 CoV 0.00880
number of training points:  14 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.016481263749379624 

LS1:  Degree 3 e_LOO 3.5795824472839923e-06 Pf_ref 0.03065 Pf 0.02235 CoV 0.02090
LS2:  Degree 3 e_LOO 0.00013177247121531427 Pf_ref 0.14828 Pf 0.06509 CoV 0.01200
number of training points:  45 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.00022042912202636353 Pf_ref 0.03205 Pf 0.02338 CoV 0.02040
LS2:  Degree 3 e_LOO 3.715089806399571e-05 Pf_ref 0.14768 Pf 0.06506 CoV 0.01200
number of training points:  46 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.0002650687644817312 Pf_ref 0.03255 Pf 0.02393 CoV 0.02020
LS2:  Degree 3 e_LOO 2.040493265214328e-05 Pf_ref 0.14884 Pf 0.07981 CoV 0.01070
number of training points:  47 -------------------------------------------------
LS1:  Degree 3 e_LOO 9.194838528816545e-05 Pf_ref 0.03117 Pf 0.0233 CoV 0.02050
LS2:  Degree 3 e_LOO 6.416148887256715e-06 Pf_ref 0.14739 Pf 0.07743 CoV 0.01090
number of training points:  48 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.000

C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\scipy\optimize\optimize.py:283: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  "minimize step, clipping to bounds", RuntimeWarning)


LS2:  Degree 3 e_LOO 1.9658281255334124e-05 Pf_ref 0.14919 Pf 0.00879 CoV 0.03360
number of training points:  27 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.0002197216941082794 Pf_ref 0.03043 Pf 0.05773 CoV 0.01280
LS2:  Degree 3 e_LOO 4.5071720238985615e-05 Pf_ref 0.14733 Pf 0.01127 CoV 0.02960
number of training points:  28 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0017871079513898244 Pf_ref 0.03103 Pf 0.07457 CoV 0.01110
LS2:  Degree 3 e_LOO 3.986480516514611e-05 Pf_ref 0.14956 Pf 0.01117 CoV 0.02980
number of training points:  29 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.000884118933772588 Pf_ref 0.03168 Pf 0.05343 CoV 0.01330
LS2:  Degree 3 e_LOO 9.363860758973731e-06 Pf_ref 0.14839 Pf 0.01182 CoV 0.02890
number of training points:  30 -------------------------------------------------
LS1:  Degree 3 e_LOO 7.593746453878194e-06 Pf_ref 0.03065 Pf 0.02675 CoV 0.01910
LS2:  Degree 4 e_LOO 6.2350

# Combined sequential training- U - LOO - 2 LS

In [8]:
number_experiments = 15    
number_active_points = 40

results_file = '_CATseq_ULOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()

         ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma) 
        
        # #################################################################################################################
        # Active training of Limit State 1 ################################################################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # #################################################################################################################
        # Active training of Limit State 2 ################################################################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        if ( len(xn) % 2 == 0 ):   #even number
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                     
            print ('LS 1')

        else:    #odd number
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2   
            print ('LS 2')
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+ sumat[voro, opt])

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), LOOCV.reshape(-1))
        
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
        
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


LS1:  Degree 1 e_LOO 1.872785478868652e-05 Pf_ref 0.03062 Pf 0.08282 CoV 0.01050
LS2:  Degree 2 e_LOO 0.0001278457126707934 Pf_ref 0.14991 Pf 0.15653 CoV 0.00730
LS 1
number of training points:  11 -------------------------------------------------
LS1:  Degree 2 e_LOO 2.258296567203248e-05 Pf_ref 0.03102 Pf 0.00056 CoV 0.13360
LS2:  Degree 2 e_LOO 0.004779841809975088 Pf_ref 0.14782 Pf 0.00349 CoV 0.05340
LS 2
number of training points:  12 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0007467505674064545 Pf_ref 0.03101 Pf 0.00025 CoV 0.20000
LS2:  Degree 3 e_LOO 0.008640573987200705 Pf_ref 0.14684 Pf 0.20673 CoV 0.00620
LS 1
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.000987578757046047 Pf_ref 0.03116 Pf 0.01519 CoV 0.02550
LS2:  Degree 3 e_LOO 0.0021130254061727336 Pf_ref 0.14886 Pf 0.20816 CoV 0.00620
LS 2
number of training points:  14 -------------------------------------------------
LS1:  Degre

LS2:  Degree 4 e_LOO 0.00016269551139018646 Pf_ref 0.14893 Pf 0.1486 CoV 0.00760
LS 2
number of training points:  44 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.0003796408971907845 Pf_ref 0.03145 Pf 0.02475 CoV 0.01990
LS2:  Degree 4 e_LOO 0.0001420744386726381 Pf_ref 0.14587 Pf 0.14672 CoV 0.00760
LS 1
number of training points:  45 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.00044469228098418655 Pf_ref 0.03163 Pf 0.02442 CoV 0.02000
LS2:  Degree 4 e_LOO 0.0001297652199944653 Pf_ref 0.14866 Pf 0.15 CoV 0.00750
LS 2
number of training points:  46 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.00021152930568903278 Pf_ref 0.03229 Pf 0.02526 CoV 0.01960
LS2:  Degree 4 e_LOO 0.00015190168788500026 Pf_ref 0.14909 Pf 0.1507 CoV 0.00750
LS 1
number of training points:  47 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.00023726546184133608 Pf_ref 0.03093 Pf 0.02319 CoV 0.02050
LS2:  De

LS2:  Degree 1 e_LOO 3.126152933243195e-05 Pf_ref 0.14861 Pf 0.15246 CoV 0.00750
LS 1
number of training points:  37 -------------------------------------------------
LS1:  Degree 4 e_LOO 5.623982824531597e-05 Pf_ref 0.03197 Pf 0.04979 CoV 0.01380
LS2:  Degree 1 e_LOO 8.693069045229585e-05 Pf_ref 0.14848 Pf 0.15251 CoV 0.00750
LS 2
number of training points:  38 -------------------------------------------------
LS1:  Degree 2 e_LOO 1.481368263177713e-05 Pf_ref 0.03119 Pf 0.03819 CoV 0.01590
LS2:  Degree 4 e_LOO 4.960527403729207e-07 Pf_ref 0.14741 Pf 0.14206 CoV 0.00780
LS 1
number of training points:  39 -------------------------------------------------
LS1:  Degree 3 e_LOO 2.4918503772057995e-06 Pf_ref 0.0314 Pf 0.03347 CoV 0.01700
LS2:  Degree 4 e_LOO 8.435449459464089e-06 Pf_ref 0.14622 Pf 0.14145 CoV 0.00780
LS 2
number of training points:  40 -------------------------------------------------
LS1:  Degree 3 e_LOO 2.459706643399375e-07 Pf_ref 0.03109 Pf 0.03398 CoV 0.01690
LS2:  De

...

number of training points:  37 -------------------------------------------------
LS1:  Degree 1 e_LOO 1.4229391017294102e-05 Pf_ref 0.03113 Pf 0.02404 CoV 0.02010
LS2:  Degree 1 e_LOO 2.0563084332382172e-05 Pf_ref 0.14869 Pf 0.14997 CoV 0.00750
LS 2
number of training points:  38 -------------------------------------------------
LS1:  Degree 1 e_LOO 3.0904665388284846e-08 Pf_ref 0.03076 Pf 0.02357 CoV 0.02040
LS2:  Degree 1 e_LOO 2.1175653023112646e-08 Pf_ref 0.1494 Pf 0.15375 CoV 0.00740
LS 1
number of training points:  39 -------------------------------------------------
LS1:  Degree 4 e_LOO 5.2639104942021905e-05 Pf_ref 0.03145 Pf 0.03087 CoV 0.01770
LS2:  Degree 3 e_LOO 4.283239475781036e-06 Pf_ref 0.14943 Pf 0.15207 CoV 0.00750
LS 2
number of training points:  40 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.00020375612567846818 Pf_ref 0.03087 Pf 0.03118 CoV 0.01760
LS2:  Degree 1 e_LOO 5.715990085390979e-05 Pf_ref 0.14876 Pf 0.1508 CoV 0.00750
LS 1
numb

number of training points:  30 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.003813387207054882 Pf_ref 0.0324 Pf 0.02735 CoV 0.01890
LS2:  Degree 2 e_LOO 0.006170828677650191 Pf_ref 0.15049 Pf 0.14592 CoV 0.00770
LS 1
number of training points:  31 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0028635788428381594 Pf_ref 0.03245 Pf 0.0258 CoV 0.01940
LS2:  Degree 2 e_LOO 0.004096946736999282 Pf_ref 0.1466 Pf 0.14238 CoV 0.00780
LS 2
number of training points:  32 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0037065314468026113 Pf_ref 0.03079 Pf 0.02459 CoV 0.01990
LS2:  Degree 2 e_LOO 0.007500544919336667 Pf_ref 0.14848 Pf 0.1447 CoV 0.00770
LS 1
number of training points:  33 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0044824862651643435 Pf_ref 0.03006 Pf 0.02372 CoV 0.02030
LS2:  Degree 1 e_LOO 0.010873146656151328 Pf_ref 0.14809 Pf 0.14355 CoV 0.00770
LS 2
number of trainin

number of training points:  23 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.000667245935365183 Pf_ref 0.02965 Pf 0.00935 CoV 0.03260
LS2:  Degree 1 e_LOO 0.00010515824961642615 Pf_ref 0.14836 Pf 0.06598 CoV 0.01190
LS 2
number of training points:  24 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0002772676755079854 Pf_ref 0.03149 Pf 0.00939 CoV 0.03250
LS2:  Degree 1 e_LOO 8.402808391092984e-05 Pf_ref 0.14696 Pf 0.06398 CoV 0.01210
LS 1
number of training points:  25 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.00012534623948995592 Pf_ref 0.03193 Pf 0.01097 CoV 0.03000
LS2:  Degree 3 e_LOO 2.9164935836425466e-07 Pf_ref 0.14913 Pf 0.06907 CoV 0.01160
LS 2
number of training points:  26 -------------------------------------------------
LS1:  Degree 2 e_LOO 4.5738125976397135e-05 Pf_ref 0.03115 Pf 0.01286 CoV 0.02770
LS2:  Degree 2 e_LOO 0.00025455803803191675 Pf_ref 0.14841 Pf 0.1578 CoV 0.00730
LS 1
numbe

number of training points:  16 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0027199066799491983 Pf_ref 0.03063 Pf 0.01232 CoV 0.02830
LS2:  Degree 2 e_LOO 0.0032008489902484623 Pf_ref 0.14572 Pf 0.11062 CoV 0.00900
LS 1
number of training points:  17 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0019194101527071328 Pf_ref 0.03181 Pf 0.02061 CoV 0.02180
LS2:  Degree 2 e_LOO 0.0024972975429536793 Pf_ref 0.14701 Pf 0.10961 CoV 0.00900
LS 2
number of training points:  18 -------------------------------------------------
LS1:  Degree 4 e_LOO 0.0013683259187782387 Pf_ref 0.0311 Pf 0.12489 CoV 0.00840
LS2:  Degree 2 e_LOO 0.001315025382416735 Pf_ref 0.14898 Pf 0.08724 CoV 0.01020
LS 1
number of training points:  19 -------------------------------------------------
LS1:  Degree 4 e_LOO 6.028671356006517e-05 Pf_ref 0.03152 Pf 0.11976 CoV 0.00860
LS2:  Degree 2 e_LOO 0.0006409586335664567 Pf_ref 0.14936 Pf 0.11213 CoV 0.00890
LS 2
number of 

number of training points:  49 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0005239339431013175 Pf_ref 0.03158 Pf 0.03012 CoV 0.01790
LS2:  Degree 1 e_LOO 0.00011870850435900728 Pf_ref 0.14933 Pf 0.14962 CoV 0.00750
LS 2
number of training points:  50 -------------------------------------------------


# Combined training - EFF - LOO - 2 LS

In [ ]:
number_experiments = 15    
number_active_points = 40

results_file = '_CATSeq_EFFLOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000
        LOOCV_1 = np.zeros(MCS_samples)
        LOOCV_2 = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
        
        #Selecting the limit state to improve (1 each loop for sequential)
        
        if ( len(xn) % 2 == 0 ):   #even number
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                     
            print ('LS 1')

        else:    #odd number
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2   
            print ('LS 2')
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+ sumat[voro, opt])

        ### Evaluating new points
        eff = EFF(meanMC.reshape(-1), LOOCV.reshape(-1) , 0 )  #(mean, variance, target) 
        
        xr = np.append(xr, MCinputs[np.argmax(eff)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff)]).reshape(-1,2)

        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))